# Welcome to CS 5242 **Homework 4**

ASSIGNMENT DEADLINE ⏰ : **19 Sept 2022** 

In this assignment, we have three parts:

1. Implement some functions in CNNs from scratch *(3 Points)*
2. Implement a CNN and train for CIFAR10 using PyTorch *(5 Points)*
3. Discussion (parametes and flops for AlexNet) *(2 Points)*

Colab is a hosted Jupyter notebook service that requires no setup to use, while providing access free of charge to computing resources including GPUs. In this semester, we will use Colab to run our experiments.

> In this assignment, We **need GPU** to training the CNN model. You may need to **choose GPU in Runtime -> Change runtime type -> Hardware accerator**

### **Grades Policy**

We have 10 points for this homework. 15% off per day late, 0 scores if you submit it 7 days after the deadline.

### **Cautions**

**DO NOT** use external libraries like PyTorch or TensorFlow in your implementation.

**DO NOT** copy the code from the internet, e.g. GitHub.

---

### **Contact**

Please feel free to contact us if you have any question about this homework or need any further information.

Slack (Recommend): Shenggan Cheng

TA Email: shenggan@comp.nus.edu.sg

> If you have not join the slack group, you can click [here](https://join.slack.com/t/cs5242ay20222-oiw1784/shared_invite/zt-1eiv24k1t-0J9EI7vz3uQmAHa68qU0aw)

## Setup

Start by running the cell below to set up all required software.

In [1]:
!pip install numpy matplotlib torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Import the neccesary library and fix seed for Python, NumPy and PyTorch.

In [2]:
import math
import random

import numpy as np
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

random.seed(0)
np.random.seed(0)
torch.manual_seed(0)

Now let's setup the GPU environment. The colab provides a free GPU to use. Do as follows:

- Runtime -> Change Runtime Type -> select `GPU` in Hardware accelerator
- Click `connect` on the top-right

After connecting to one GPU, you can check its status using `nvidia-smi` command.

In [3]:
!nvidia-smi

torch.cuda.is_available()

Mon Sep 26 06:33:38 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   51C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

True

Everything is ready, you can move on and ***Good Luck !*** 😃

## Implement functions in CNNs from scratch

In this section, you need to implement some functions commonly used in CNNs, including convolution, pooling, etc. 

We will compare the computational results of your implemented version with those of pytorch, expecting that the error between the correct implementation and pytorch will be very small.

NOTE: 

1. Implement these functions from scratch, **without** using any neural network libraries. Use linear algebra libraries in python is ok.

2. The performance of the function is not included in this scoring, You just need to pay attention to the correctness of your implementation.

### Step 1
Given a 32x32 pixels, 3 channels input, get a torch tensor with torch.randn().

In [4]:
batch_size = 2
x = torch.randn(batch_size, 3, 32, 32)

### Step 2

For each following functions in the list, get the output tensor "torch_xxx_out" with input as x:

In [5]:
torch_max_pool = nn.MaxPool2d(kernel_size=2,
                              stride=1,
                              padding=0,
                              dilation=1,
                              return_indices=False,
                              ceil_mode=False)
torch_avg_pool = nn.AvgPool2d(kernel_size=2,
                              stride=1,
                              padding=0,
                              ceil_mode=False,
                              count_include_pad=True,
                              divisor_override=None)
torch_conv = nn.Conv2d(in_channels=3,
                       out_channels=6,
                       kernel_size=3,
                       stride=1,
                       padding=0,
                       dilation=1,
                       groups=1,
                       bias=True,
                       padding_mode='zeros')
torch_norm = nn.BatchNorm2d(3)

In [6]:
torch_sigmoid_out = torch.sigmoid(x, out=None)
tmp_tensor = torch.randint(3, (batch_size,))
torch_cross_entropy_out = F.cross_entropy(x[::, ::, 0, 0], tmp_tensor)

In [7]:
torch_max_pool_out = torch_max_pool(x)
torch_avg_pool_out = torch_avg_pool(x)
torch_conv_out = torch_conv(x)
torch_norm_out = torch_norm(x)

### Step 3

Implement these functions from scratch, without using any neural network libraries. Use linear algebra libraries in python is ok. Output your tensors as "my_xxx_out".

In [8]:
def my_max_pool(x, kernel_size, stride, padding):
    """
    Args:
        x: torch tensor with size (N, C_in, H_in, W_in),
        kernel_size: size of the window to take a max over, 
        stride: stride of the window,
        padding: implicit zero padding to be added on both sides,
        
    Return:
        y: torch tensor of size (N, C_out, H_out, W_out).
    """

    y = None
    # === Complete the code (0.5')
    N, C_in, H_in, W_in = x.shape
    H_out = (H_in + 2 * padding - kernel_size) // stride + 1
    W_out = (W_in + 2 * padding - kernel_size) // stride + 1
    np_y = np.empty((N, C_in, H_out, W_out))

    padded_x = np.zeros((N, C_in, H_in + 2 * padding, W_in + 2 * padding))
    padded_x[:, :, padding:padding + H_in, padding:padding + W_in] = x
    for i in range(H_out):
        for j in range(W_out):
            np_y[:, :, i, j] = np.max(padded_x[:, :, i * stride:i * stride + kernel_size, j * stride:j * stride + kernel_size], axis=(2, 3))
    y = torch.from_numpy(np_y)
    # === Complete the code
    return y

In [9]:
def my_avg_pool(x, kernel_size, stride, padding):
    """
    Args:
        x: torch tensor with size (N, C_in, H_in, W_in),
        kernel_size: size of the window, 
        stride: stride of the window,
        padding: implicit zero padding to be added on both sides,
        
    Return:
        y: torch tensor of size (N, C_out, H_out, W_out).
    """

    y = None
    # === Complete the code (0.5')
    N, C_in, H_in, W_in = x.shape
    H_out = (H_in + 2 * padding - kernel_size) // stride + 1
    W_out = (W_in + 2 * padding - kernel_size) // stride + 1
    np_y = np.empty((N, C_in, H_out, W_out))

    padded_x = np.zeros((N, C_in, H_in + 2 * padding, W_in + 2 * padding))
    padded_x[:, :, padding:padding + H_in, padding:padding + W_in] = x
    for i in range(H_out):
        for j in range(W_out):
            np_y[:, :, i, j] = np.mean(padded_x[:, :, i * stride:i * stride + kernel_size, j * stride:j * stride + kernel_size], axis=(2, 3))
    y = torch.from_numpy(np_y)
    # === Complete the code
    return y

In [10]:
def my_conv(x, in_channels, out_channels, kernel_size, stride, padding, weight, bias):
    """
    Args:
        x: torch tensor with size (N, C_in, H_in, W_in),
        in_channels: number of channels in the input image, it is C_in;
        out_channels: number of channels produced by the convolution;
        kernel_size: size of onvolving kernel, 
        stride: stride of the convolution,
        padding: implicit zero padding to be added on both sides of each dimension,
        
    Return:
        y: torch tensor of size (N, C_out, H_out, W_out)
    """

    y = None
    # === Complete the code (0.5')
    N, C_in, H_in, W_in = x.shape
    H_out = (H_in + 2 * padding - kernel_size) // stride + 1
    W_out = (W_in + 2 * padding - kernel_size) // stride + 1
    np_y = np.empty((N, out_channels, H_out, W_out))

    padded_x = np.zeros((N, in_channels, H_in + 2 * padding, W_in + 2 * padding))
    padded_x[:, :, padding:padding + H_in, padding:padding + W_in] = x
    for i in range(H_out):
        for j in range(W_out):
          array_to_pool = padded_x[:, :, i*stride : i*stride + kernel_size, j*stride : j*stride + kernel_size]
          reshape_array = array_to_pool.reshape(N, -1)
          reshape_weight = weight.reshape(out_channels, -1)
          np_y[:, :, i, j] = np.matmul(reshape_array, reshape_weight.T.detach().numpy()) + bias.detach().numpy()
    y = torch.from_numpy(np_y)
    # === Complete the code
    return y

In [11]:
def my_batchnorm(x, num_features, eps):
    """
    Args:
        x: torch tensor with size (N, C, H, W),
        num_features: number of features in the input tensor, it is C;
        eps: a value added to the denominator for numerical stability. Default: 1e-5
        
    Return:
        y: torch tensor of size (N, C, H, W)
    """

    y = torch.empty_like(x)
    # === Complete the code (0.5')
    gamma = np.ones((1, num_features, 1, 1))
    beta = np.zeros((1, num_features, 1, 1))
    mean = np.mean(x.detach().numpy(), axis=(0, 2, 3), keepdims=True)
    var = np.var(x.detach().numpy(), axis=(0, 2, 3), keepdims=True)
    np_y = gamma * (x.detach().numpy() - mean) / np.sqrt(var + eps) + beta
    y = torch.from_numpy(np_y)
    # === Complete the code
    return y

In [12]:
def my_sigmoid(x):
    """
    Args:
        x: torch tensor with any size

    Return:
        y: the logistic sigmoid function of x
    """
    y = None
    # === Complete the code (0.5')
    y = torch.from_numpy(1 / (1 + np.exp(-x.detach().numpy())))
    # === Complete the code
    return y

In [13]:
def my_cross_entropy(p, y):
    """
    Args:
        p: torch tensor with size of (N, C),
        y (int): torch tensor with size of (N), the values range from 0 to C-1

    Return:
        loss: the cross_entropy of predicted values p and target y.
    """
    loss = None
    # === Complete the code (0.5')
    softmax_probs = np.exp(p.detach().numpy()) / np.sum(np.exp(p.detach().numpy()), axis=1, keepdims=True)
    loss = torch.from_numpy(np.asarray(np.mean(-np.log(softmax_probs[range(p.shape[0]), y]))))
    # === Complete the code
    return loss

In [14]:
my_max_pool_out = my_max_pool(x, kernel_size=2, stride=1, padding=0)
my_avg_pool_out = my_avg_pool(x, kernel_size=2, stride=1, padding=0)
my_conv_out = my_conv(x,
                      in_channels=3,
                      out_channels=6,
                      kernel_size=3,
                      stride=1,
                      padding=0,
                      weight=torch_conv.weight,
                      bias=torch_conv.bias)
my_norm_out = my_batchnorm(x, num_features=3, eps=1e-5)

In [15]:
my_sigmoid_out = my_sigmoid(x)
my_cross_entropy_out = my_cross_entropy(x[::, ::, 0, 0], tmp_tensor)

### Step 4

Compare and show that "torch_xxx_out" and "my_xxx_out" are equal up to small numerical errors.

In [16]:
print(F.mse_loss(my_max_pool_out, torch_max_pool_out))
print(F.mse_loss(my_avg_pool_out, torch_avg_pool_out))
print(F.mse_loss(my_conv_out, torch_conv_out))
print(F.mse_loss(my_norm_out, torch_norm_out))

tensor(0., dtype=torch.float64)
tensor(4.2491e-16, dtype=torch.float64)
tensor(3.1710e-15, dtype=torch.float64, grad_fn=<MseLossBackward0>)
tensor(2.2439e-15, dtype=torch.float64, grad_fn=<MseLossBackward0>)


In [17]:
print(F.mse_loss(my_sigmoid_out, torch_sigmoid_out))
print(F.mse_loss(my_cross_entropy_out, torch_cross_entropy_out))

tensor(3.6253e-16)
tensor(0.)


## Train CNNs on CIFAR-10 dataset

Implement a CNN and train for CIFAR10 with these definitions:

1. cA-B = Conv2d with input A channels, output B channels - kernel size 3x3, stride (1,1), padding with zeros to keep image size constant, followed by ReLU;

2. mp = maxpool2d kernel size 2x2, stride (2,2);

3. bn = batchnorm2d with affine=False (i.e. non learning batch norm);

4. fcA-B = nn.linear with input A nodes, output B nodes;

5. aap = adaptive average pooling.

Use the definition to make the architecture c3-16 -> c16-16 -> mp -> c16-32 -> c32-32 -> mp -> c32-64 -> c64-64 -> mp -> c64-128 -> c128-128 -> aap -> flatten -> fc128-10 -> cross entropy loss. Adjust learning rate, batch size and other hyper parameters to make classification results **> 75%**.

In [18]:
# === Complete the code (1')
num_epoch = 20 # TODO: please define the number of epoch here.
batch_size = 64 # TODO: please fill the batch size here.
# === Complete the code

In [19]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data',
                                        train=True,
                                        download=True,
                                        transform=transform)
trainloader = torch.utils.data.DataLoader(trainset,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          num_workers=1)

testset = torchvision.datasets.CIFAR10(root='./data',
                                       train=False,
                                       download=True,
                                       transform=transform)
testloader = torch.utils.data.DataLoader(testset,
                                         batch_size=batch_size,
                                         shuffle=False,
                                         num_workers=1)

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [20]:
# Creating a CNN model
class CNN(nn.Module):
    
    def __init__(self, num_classes):
        super(CNN, self).__init__()
       
        # === Complete the code (1.5')
        self.c3_16 = nn.Conv2d(3, 16, (3,3), 1, 1)
        self.c16_16 = nn.Conv2d(16, 16, (3,3), 1, 1)
        self.c16_32 = nn.Conv2d(16, 32, (3,3), 1, 1)
        self.c32_32 = nn.Conv2d(32, 32, (3,3), 1, 1)
        self.c32_64 = nn.Conv2d(32, 64, (3,3), 1, 1)
        self.c64_64 = nn.Conv2d(64, 64, (3,3), 1, 1)
        self.c64_128 = nn.Conv2d(64, 128, (3,3), 1, 1)
        self.c128_128 = nn.Conv2d(128, 128, (3,3), 1, 1)
        self.relu = nn.ReLU()

        self.mp = nn.MaxPool2d(2, 2)
        self.aap = nn.AdaptiveAvgPool2d((1,1))
        self.flatten = nn.Flatten()
        self.fc128_10 = nn.Linear(128, num_classes)
        
        # === Complete the code
        
    def forward(self, x):
        # === Complete the code (1.5')
        out = self.relu(self.c3_16(x))
        out = self.relu(self.c16_16(out))
        out = self.mp(out)
        out = self.relu(self.c16_32(out))
        out = self.relu(self.c32_32(out))
        out = self.mp(out)
        out = self.relu(self.c32_64(out))
        out = self.relu(self.c64_64(out))
        out = self.mp(out)
        out = self.relu(self.c64_128(out))
        out = self.relu(self.c128_128(out))
        out = self.aap(out)
        out = self.flatten(out)
        out = self.fc128_10(out)
        # === Complete the code
        return out

In [21]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model = CNN(10).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0005)

In [22]:
for epoch in range(num_epoch):

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):

        inputs, labels = data[0].to(device), data[1].to(device)
        # === Complete the code (1')
        optimizer.zero_grad()
        y_pred = model.forward(inputs)
        loss = criterion(y_pred, labels)
        loss.backward()
        optimizer.step()
        # === Complete the code

        running_loss += loss.item()
        if (i + 1) % 128 == 0:
            print('epoch {:3d} | {:5d} batches loss: {:.4f}'.format(epoch, i + 1, running_loss/128))
            running_loss = 0.0

print('Finished Training')

epoch   0 |   128 batches loss: 2.1503
epoch   0 |   256 batches loss: 1.9540
epoch   0 |   384 batches loss: 1.8024
epoch   0 |   512 batches loss: 1.7085
epoch   0 |   640 batches loss: 1.6382
epoch   0 |   768 batches loss: 1.5766
epoch   1 |   128 batches loss: 1.5202
epoch   1 |   256 batches loss: 1.4625
epoch   1 |   384 batches loss: 1.4347
epoch   1 |   512 batches loss: 1.3864
epoch   1 |   640 batches loss: 1.3194
epoch   1 |   768 batches loss: 1.2890
epoch   2 |   128 batches loss: 1.2686
epoch   2 |   256 batches loss: 1.2431
epoch   2 |   384 batches loss: 1.2300
epoch   2 |   512 batches loss: 1.1967
epoch   2 |   640 batches loss: 1.1782
epoch   2 |   768 batches loss: 1.1417
epoch   3 |   128 batches loss: 1.1243
epoch   3 |   256 batches loss: 1.1070
epoch   3 |   384 batches loss: 1.0907
epoch   3 |   512 batches loss: 1.0766
epoch   3 |   640 batches loss: 1.0439
epoch   3 |   768 batches loss: 1.0600
epoch   4 |   128 batches loss: 1.0168
epoch   4 |   256 batches

In [23]:
dataiter = iter(testloader)
images, labels = dataiter.next()

In [24]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        #images, labels = data
        images, labels = data[0].to(device), data[1].to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 10000 test images: {100 * correct // total} %')

Accuracy of the network on the 10000 test images: 76 %


## Discussion (2 points)

Calculate Parameters and FLOPs(Floating point operations) of **AlexNet** and analyse the ratio of the number of parameters and the amount of calculations for different layers in AlexNet.

Hint:

1. You can refer https://pytorch.org/vision/stable/_modules/torchvision/models/alexnet.html for architecture of AlexNet.
2. You only need to make estimates and do not need to perform rigorous calculations, (e.g. only consider the FLOPs of the convolution and FC in AlexNet model)
3. Because Multiply Accumulate (MAC) operations are performed on the hardware, it is possible to simply consider only the number of multiplications when considering the number of operations when calculating FLOPs.

The AlexNet architecture is as follows: <br>
```python
self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.avgpool = nn.AdaptiveAvgPool2d((6, 6))
        self.classifier = nn.Sequential(
            nn.Dropout(p=dropout),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(p=dropout),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes),
        )
```

* Number of Parameters
  * The number of parameters is calculated by the sum of the weights and the bias in each layer.
  * The number of weight parameters are the number of conv/fc kernels * size of kernel * number of input channels
  * The bias has the same size as the number of kernels.
  * Max pooling and normalisation layers do not have any parameters associated with them.

* Number of Floating Point Operations (FLOPs)
  * The number of FLOPs is calculated as the product of the dimensions of the weights by the size of the outputs (as these represent the number of times the kernel performs its multiplications).
  * FLOPs for max pooling and normalisation layers are not calculated.

| Layer | Stride | Padding |  Input Size | Kernel Size | Output Size | Weights Calculation | Weights | Bias | # Parameters | FLOPs Calculation | # FLOPs | # Parameters / # FLOPs
| --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- |
| Conv | 4 | 2 | [224x224x3] | [11x11x3] | [55x55x64] | 64 * [11x11x3] | 23232 | 64 | 23296 | [55x55]x64x[11x11x3]+64x3 | 70276992 | 3.3 * e-3
| MaxPool | 2 | 0 | [55x55x64] | [3x3] | [27x27x64] | - | - | - | 0 | - | 0 | -
| Conv | 1 | 2 | [27x27x64] | [5x5x64] | [27x27x192] | 192 * [5x5x64] | 307200 | 192 | 307392 | [27x27]x192x[5x5x64]+192x64 | 223961088 | 1.3 * e-3
| MaxPool | 2 | 0 | [27x27x192] | [3x3] | [13x13x192] | - | - | - | 0 | - | 0 | -
| Conv | 1 | 1 | [13x13x192] | [3x3x192] | [13x13x384] | 384 * [3x3x192] | 663552 | 384 | 663936 | [13x13]x384x[3x3x192]+192x384 | 112214016 | 5.9 * e-3
| Conv | 1 | 1 | [13x13x384] | [3x3x384] | [13x13x256] | 256 * [3x3x384] | 884736 | 256 | 884992 | [13x13]x256x[3x3x384]+384x256 | 149618688 | 5.9 * e-3
| Conv | 1 | 1 | [13x13x256] | [3x3x256] | [13x13x256] | 256 * [3x3x256] | 589824 | 256 | 590080 | [13x13]x256x[3x3x256]+256x256 | 99745792 | 5.9 * e-3
| MaxPool | 2 | 0 | [13x13x256] | [3x3] | [6x6x256] | - | - | - | 0 | - | 0 | -
| FC | - | - | [6x6x256] | - | [4096x1] | 4096 * [6x6x256] | 37748736 | 4096 | 37752832 | [4096x1]x[6x6x256] + 4096 | 37752832 | 1
| FC | - | - | [4096x1] | - | [4096x1] | 4096 * [4096x1] | 16777216 | 4096 | 16781312 | [4096x1]x[4096x1] + 4096 | 16781312 | 1
| FC | - | - | [4096x1] | - | [1000x1] | 1000 * [4096x1] | 4096000 | 1000 | 4097000 | [4096x1]x[1000x1] + 1000 | 4097000 | 1

The total number of parameters is 61100840 ~ 61 million. <br>
The total number of FLOPs is 714447720 ~ 714 million.

* Convolutional Layers
  * For a Convolutional Layer, the number of FLOPs is proportional to the number of parameters multiplied by the 2 dimensional size of the output.
  * The magnitude of all ratios for convolutional layers is approximately around 4e-3
  * We can see that a smaller kernel size leads to a larger ratio of parameters to FLOPs, because it increases the number of multiplication operations required.
* Fully Connected Layers
  * For Fully Connected Layers, we can see that the number of parameters is equal to the number of Floating Point Operations (FLOPs).
  * This is expected as the presence of no external kernel makes each parameter contribute to one operation.
* From these, it appears that the number of FLOPs is only larger when the layer creates newer transformations/complexities (features) from our inputs